In [1]:
pip install python-docx


In [2]:
pip install faiss-gpu

In [3]:
from docx import Document
import re
from transformers import AutoModel, AutoTokenizer
import numpy as np
import torch
import faiss

Loading unstructure data from word document:

In [4]:
def read_docx(file_path):
    # Load the document
    doc = Document(file_path)
    full_text = []

    # Iterate through each paragraph in the document
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all text into a single string
    return '\n'.join(full_text), doc.paragraphs

# Example usage
file_path = 'grazinimu-taisykles.docx'
document_text, paragraphs = read_docx(file_path)


Preprocessing raw text to paragraphs and storing paragraphs into list:

In [5]:
contexts = [paragraphs[i].text for i in range(len(paragraphs)) if len(paragraphs[i].text) != 0]

questions = ["kaip grąžinti prekę",
             "kokios yra grąžinimo taisyklės",
             "kokie yra prekių pažeidimo tipai",
             "prekė buvo pažeista transportavimo metu"]

Load google T5 language model and tokenize text data:

In [6]:
model_name = "sentence-transformers/LaBSE"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
context_inputs = tokenizer(contexts)
questions_inputs = tokenizer(questions)

context_inputs_pt = tokenizer(contexts, return_tensors="pt", truncation=True, padding=True)
questions_inputs_pt = tokenizer(questions, return_tensors="pt", truncation=True, padding=True)

Defining matching function which retrieves the context paragraph with most matching tokens:

In [8]:
def match_context(question_id, contexts_ids):
  question_set = set(question_id)
  counts = []

  for ids in contexts_ids:
    common_elements = question_set & set(ids)
    counts.append(len(common_elements))

  predicted_index = np.argmax(counts)
  return int(predicted_index)


Defining matching function which retrieves the context paragraph by measuring cosine similarit scores of T5 embeddings of last hidden state:

In [9]:
def get_embeddings(tokenized_inputs):
  with torch.no_grad():
    outputs = model(**tokenized_inputs)

  embeddings = outputs.last_hidden_state
  attention_mask = tokenized_inputs['attention_mask']
  extended_attention_mask = attention_mask.unsqueeze(-1).expand(embeddings.size())
  masked_embeddings = embeddings * extended_attention_mask
  summed_embeddings = torch.sum(masked_embeddings, dim=1)
  summed_mask = torch.clamp(extended_attention_mask.sum(1), min=1e-9)
  mean_pooled_embeddings = summed_embeddings / summed_mask

  return mean_pooled_embeddings

def semantic_search(q_embedding, c_embeddings):
  dim = c_embeddings.shape[1]
  index = faiss.IndexFlatIP(dim)

  index.add(c_embeddings)

  scores, corpus_ids = index.search(q_embedding.reshape(1, -1), k=1)

  return corpus_ids[0][0]

Results:

Matching by token count:

In [10]:
predicted_context = match_context(questions_inputs['input_ids'][0], context_inputs['input_ids'])
print(f"klausimas: {questions[0]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija : {contexts[predicted_context]}")

klausimas: kaip grąžinti prekę 
 atspėtas kontekstas (37) 
 surasta informacija : Grąžinamą prekę būtina užregistruoti adresu http://www.accdistribution.net/_al/lt-lt/returns užpildžius reikiamus laukus. Grąžinama prekė turi būti tvarkinga: nepažeista pakuotė, prekė švari, nesubraižyta, prekės serijos numeriai ant pakuotės ir prekės turi sutapti, prekė turi būti pilnos komplektacijos, programinė įranga (jei tokia yra) turi būti tokia pati kaip naujos prekės. „By Order“ (Užsakoma iš anksto) požymiu pažymėtų arba pagal specialų kliento užsakymą pristatytų prekių grąžinimai nepriimami (tai nustačius prekės bus grąžintos pirkėjui).


In [11]:
predicted_context = match_context(questions_inputs['input_ids'][1], context_inputs['input_ids'])
print(f"klausimas: {questions[1]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija : {contexts[predicted_context]}")

klausimas: kokios yra grąžinimo taisyklės 
 atspėtas kontekstas (28) 
 surasta informacija : Vartotojas turi galimybę susipažinti su grąžinimo taisyklėmis ir jei grąžinimas registruotas per grąžinimo tipą, kuris yra apmokestinamas – mato preliminarią paskaičiuotą grąžinimo kainą. Jei visos sąlygos tinka, vartotojas turi pažymėti sutikimą su grąžinimo taisyklėmis (7)  ir spausti mygtuką “Grąžinti” (8).


In [12]:
predicted_context = match_context(questions_inputs['input_ids'][2], context_inputs['input_ids'])
print(f"klausimas: {questions[2]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija: {contexts[predicted_context]}")

klausimas: kokie yra prekių pažeidimo tipai 
 atspėtas kontekstas (37) 
 surasta informacija: Grąžinamą prekę būtina užregistruoti adresu http://www.accdistribution.net/_al/lt-lt/returns užpildžius reikiamus laukus. Grąžinama prekė turi būti tvarkinga: nepažeista pakuotė, prekė švari, nesubraižyta, prekės serijos numeriai ant pakuotės ir prekės turi sutapti, prekė turi būti pilnos komplektacijos, programinė įranga (jei tokia yra) turi būti tokia pati kaip naujos prekės. „By Order“ (Užsakoma iš anksto) požymiu pažymėtų arba pagal specialų kliento užsakymą pristatytų prekių grąžinimai nepriimami (tai nustačius prekės bus grąžintos pirkėjui).


In [13]:
predicted_context = match_context(questions_inputs['input_ids'][3], context_inputs['input_ids'])
print(f"klausimas: {questions[3]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija: {contexts[predicted_context]}")

klausimas: prekė buvo pažeista transportavimo metu 
 atspėtas kontekstas (50) 
 surasta informacija: Galima grąžinti prekes, kurios buvo apgadintos transportavimo metu, be atskiro vadybininko grąžinimo patvirtinimo. Vykdant šio tipo grąžinimą, tolesniuose vedlio žingsniuose bus prašoma nurodyti pažeidimus įrodančius dokumentus: vežėjo pažeidimo aktą, nuotraukas. Jokie papildomi mokesčiai nebus skaičiuojami. Prekės grąžinimas nepriimamas jei nebuvo laikomasi prekių pristatymo tvarkoje nurodytų siuntų priėmimo sąlygų (https://accdistribution.net/_al/lt-lt/page/products-delivery).


matching by cosine similarity:

In [14]:
c_embeddings = get_embeddings(context_inputs_pt)
q_embeddings = get_embeddings(questions_inputs_pt)

In [15]:
predicted_context = semantic_search(np.array(q_embeddings[0]), c_embeddings.numpy())
print(f"klausimas: {questions[0]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija: {contexts[predicted_context]}")

klausimas: kaip grąžinti prekę 
 atspėtas kontekstas (2) 
 surasta informacija: Inicijuoti prekės grąžinimą galima dviem būdais: 


In [16]:
predicted_context = semantic_search(np.array(q_embeddings[1]), c_embeddings.numpy())
print(f"klausimas: {questions[1]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija: {contexts[predicted_context]}")

klausimas: kokios yra grąžinimo taisyklės 
 atspėtas kontekstas (36) 
 surasta informacija: Grąžinimų taisyklės


In [17]:
predicted_context = semantic_search(np.array(q_embeddings[2]), c_embeddings.numpy())
print(f"klausimas: {questions[2]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija: {contexts[predicted_context]}")

klausimas: kokie yra prekių pažeidimo tipai 
 atspėtas kontekstas (41) 
 surasta informacija: Pažeidimų tipai:


In [18]:
predicted_context = semantic_search(np.array(q_embeddings[3]), c_embeddings.numpy())
print(f"klausimas: {questions[3]}","\n",
      f"atspėtas kontekstas ({predicted_context}) \n",
      f"surasta informacija: {contexts[predicted_context]}")

klausimas: prekė buvo pažeista transportavimo metu 
 atspėtas kontekstas (14) 
 surasta informacija: Transportavimo metu pažeista prekė (su pažeidimo aktu, nuotraukomis)
